In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from sklearn import set_config; set_config(display='diagram')
from package.etl import *
from package.train_family import TrainFamily
import pandas as pd
import numpy as np
import requests
import joblib
import json

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

import matplotlib.pyplot as plt

# RNN

In [3]:
path = 'gs://business-case/'

In [4]:
df_transactions = pd.read_csv(path+'transactions.csv')
df_transactions 

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


In [6]:
df_train = pd.read_csv(path+'train.csv')

In [7]:
df = pd.merge(df_train, df_transactions, how='left', on=['date','store_nbr'])
df

,id,date,store_nbr,family,sales,onpromotion,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,NaN
1,1,2013-01-01,1,BABY CARE,0.000,0,NaN
2,2,2013-01-01,1,BEAUTY,0.000,0,NaN
3,3,2013-01-01,1,BEVERAGES,0.000,0,NaN
4,4,2013-01-01,1,BOOKS,0.000,0,NaN
...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,2155.0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,2155.0
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,2155.0
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,2155.0


In [8]:
df.transactions.replace(np.nan, 0, inplace=True)

In [9]:
drop = ['onpromotion']
features = ['ds','store_nbr','family']

df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = 1
df['ds'] = pd.to_datetime(df[["year", "month", "day"]])
drop.extend(['id','date','year','month','day'])
df = df.drop(columns=drop)
df = df.groupby(features).sum().reset_index()

In [21]:
df.query("ds=='2013-01-01' & store_nbr==1")

,ds,store_nbr,family,sales,transactions
0,2013-01-01,1,AUTOMOTIVE,68.000000,47471.0
1,2013-01-01,1,BABY CARE,0.000000,47471.0
2,2013-01-01,1,BEAUTY,45.000000,47471.0
3,2013-01-01,1,BEVERAGES,28777.000000,47471.0
4,2013-01-01,1,BOOKS,0.000000,47471.0
5,2013-01-01,1,BREAD/BAKERY,8559.680975,47471.0
6,2013-01-01,1,CELEBRATION,0.000000,47471.0
7,2013-01-01,1,CLEANING,20388.000000,47471.0
8,2013-01-01,1,DAIRY,11789.000000,47471.0
9,2013-01-01,1,DELI,3425.182000,47471.0


In [ ]:
df.query("date=='2013-01-02' & store_nbr==1")

In [ ]:
df_train.query("date=='2013-01-02' & store_nbr==1")

# Linear Baseline

In [ ]:
X_train, y_train, X_predict, pipe, preprocessor = TrainFamily()
X_train.shape, y_train.shape

In [ ]:
X_train

In [ ]:
X_train.groupby(['city','store_nbr']).sum().reset_index()

In [ ]:
job = joblib.load('model_family.joblib')
job

In [ ]:
job2 = joblib.load('/home/yuka/code/mrdaraujo/business_case_869/model_family.joblib')
job2

In [ ]:
job.score(X_train,y_train), job2.score(X_train,y_train)

In [ ]:
X = pd.DataFrame(dict(year=[2020,2020],
             month=[1,2],
             city=['Quito','Quito'],
             store_nbr=[1,1],
             family=['AUTOMOTIVE','AUTOMOTIVE']
            ))
X

In [ ]:
job.predict(X), job2.predict(X)

In [ ]:
model = LinearRegression()
X_train_transformed = preprocessor.fit_transform(X_train)
cv_results = cross_validate(model, X_train_transformed, y_train, cv=5, 
                            scoring=['max_error',
                                     'r2', 
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error']
                           )
pd.DataFrame(cv_results)

In [ ]:
cv_results['test_r2'].mean(), cv_results['test_neg_mean_squared_error'].mean()

# FB Prophet (aborted)

In [ ]:
#def DataMerging(csv_left, csv_stores, drop, features):
#    ''' function for merging stores dataset to train/test '''
#    csv_stores = csv_stores[['store_nbr','city']]
#    df = pd.merge(csv_left, csv_stores, how='left', on='store_nbr')
#    df['date'] = pd.to_datetime(df['date'])
#    df['year'] = df['date'].dt.year
#    df['month'] = df['date'].dt.month
#    df['day'] = 1
#    df['ds'] = pd.to_datetime(df[["year", "month", "day"]])
#    drop.extend(['id','date','year','month','day'])
#    df = df.drop(columns=drop)
#    df = df.groupby(features).sum().reset_index()

#    return df

#def DataTrain(drop, features):
#    ''' function for for getting X and y train sets'''
#    path = 'gs://business-case/'
#    csv_left = pd.read_csv(path+'train.csv')
#    csv_stores = pd.read_csv(path+"stores.csv")
#    df = DataMerging(csv_left, csv_stores, drop, features)
#    #df.loc[df['sales']==0,'sales'] = 0.1
#    #df['sales'] = np.log(df['sales'])
#    df['y'] = df['sales']
#    X_train = df.drop(columns='sales')
#    #y_train = df['sales']
    

#    return X_train

In [ ]:
#drop = ['onpromotion','city', 'store_nbr', 'family']
#features = ['ds']
#df = DataTrain(drop, features)

In [ ]:
#df.head()

In [ ]:
#plt.figure(figsize=(12, 8))
#plt.grid(linestyle='-.')

#plt.plot(df['ds'], df['y'], 'b')
#plt.show()

# Random Forest Regressor (aborted)

In [ ]:
#model = RandomForestRegressor()

#cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse=False)

#preprocessor = ColumnTransformer([
#                            ('cat_tr', cat_transformer, ['year', 'month', 'city', 'store_nbr', 'family'])
#                        ],remainder='passthrough')

#pipeline = make_pipeline(preprocessor, model)
#pipeline.fit(X_train, y_train)
#joblib.dump(pipeline, 'model_rf_family.joblib')

In [ ]:
#pipeline.score(X_train,y_train)

In [ ]:
#X_train_transformed = preprocessor.fit_transform(X_train)
#cv_results = cross_validate(model, X_train_transformed, y_train, cv=5, 
#                            scoring=['max_error',
#                                     'r2', 
#                                     'neg_mean_absolute_error',
#                                     'neg_mean_squared_error']
#                           )
#pd.DataFrame(cv_results)

In [ ]:
#cv_results['test_neg_mean_squared_error'].mean()